# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from IPython.display import display

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [2]:
df = pd.read_csv("output_data/cities_output.csv")
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mahebourg,-35.809229,71.779330,73.26,74,16,9.53,MU,1654628470
1,1,east london,-56.502890,47.439327,64.38,60,18,6.13,ZA,1654628544
2,2,erenhot,43.676734,111.158750,60.42,32,7,8.46,CN,1654628544
3,3,lira,2.736162,33.086280,65.39,93,100,4.59,UG,1654628544
4,4,illoqqortoormiut,70.133680,-21.673351,46.47,95,5,3.27,NZ,1654628544


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
# Rename columns to get rid of spaces
df = df.rename(columns={"Max Temp" : "Max_Temp"})

# Create a DataFrame of my preferred climate
vacation_df = df[(df.Humidity > 70) & (df.Cloudiness  < 10) & (df.Max_Temp > 60)]

vacation_df

,Unnamed: 0,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,40,yellowknife,-59.660396,45.039541,61.09,72,0,11.18,KZ,1654628259
43,43,chifeng,-65.128975,7.699145,81.12,77,4,5.99,US,1654628554
45,45,kiunga,-1.067724,149.241806,85.33,75,4,13.89,IN,1654628555
63,63,auki,20.956211,72.066456,76.05,72,2,15.10,CV,1654628559
77,77,cherskiy,3.085813,60.267017,83.16,80,0,11.30,PK,1654628563
127,127,xining,14.715653,91.811492,81.64,86,4,18.41,PK,1654628481
132,132,mineiros,10.719031,-98.502068,74.03,82,1,0.81,PT,1654628576
167,167,klaksvik,81.546517,-84.890339,75.56,77,0,6.31,EG,1654628585
182,182,yulara,24.580796,130.314793,62.89,82,0,5.75,ZA,1654628362
187,187,marsa matruh,-3.803527,52.472364,75.42,80,0,10.40,LY,1654628590


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in vacation_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params, verify=False)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        vacation_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")



C:\Users\bryantim\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Missing field/result... skipping.


C:\Users\bryantim\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Missing field/result... skipping.


C:\Users\bryantim\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

C:\Users\bryantim\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Missing field/result... skipping.


C:\Users\bryantim\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Missing field/result... skipping.
Missing field/result... skipping.


C:\Users\bryantim\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\bryantim\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [40]:
vacation_df

,Unnamed: 0,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
40,40,yellowknife,-59.660396,45.039541,61.09,72,0,11.18,KZ,1654628259,NaN
43,43,chifeng,-65.128975,7.699145,81.12,77,4,5.99,US,1654628554,NaN
45,45,kiunga,-1.067724,149.241806,85.33,75,4,13.89,IN,1654628555,NaN
63,63,auki,20.956211,72.066456,76.05,72,2,15.10,CV,1654628559,Aamani Beach Villa
77,77,cherskiy,3.085813,60.267017,83.16,80,0,11.30,PK,1654628563,NaN
127,127,xining,14.715653,91.811492,81.64,86,4,18.41,PK,1654628481,NaN
132,132,mineiros,10.719031,-98.502068,74.03,82,1,0.81,PT,1654628576,NaN
167,167,klaksvik,81.546517,-84.890339,75.56,77,0,6.31,EG,1654628585,NaN
182,182,yulara,24.580796,130.314793,62.89,82,0,5.75,ZA,1654628362,NaN
187,187,marsa matruh,-3.803527,52.472364,75.42,80,0,10.40,LY,1654628590,NaN


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
